# 03 - modelo con preprocesado imputación Estrato 2 y SVM

In [ ]:
# =============================================
# 1. Librerías
# =============================================
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

# =============================================
# 2. Cargar datos
# =============================================
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print("Shape train:", train.shape)
print("Shape test :", test.shape)

# =============================================
# 3. Selección de columnas
# =============================================
cols = [
    'ESTU_VALORMATRICULAUNIVERSIDAD',
    'FAMI_ESTRATOVIVIENDA',
    'FAMI_TIENEINTERNET',
    'FAMI_EDUCACIONPADRE',
    'FAMI_EDUCACIONMADRE',
    'ESTU_HORASSEMANATRABAJA',
    'ESTU_PAGOMATRICULAPROPIO',
    'ESTU_PRGM_DEPARTAMENTO'
]

train = train[['ID'] + cols + ['RENDIMIENTO_GLOBAL']]
test = test[['ID'] + cols]

# =============================================
# 4. Imputación de faltantes (con Estrato 2)
# =============================================
for col in cols:
    modo = 'Estrato 2' if col == 'FAMI_ESTRATOVIVIENDA' else train[col].mode()[0]
    train[col] = train[col].fillna(modo)
    test[col] = test[col].fillna(modo)

# =============================================
# 5. One-hot encoding
# =============================================
def one_hot(df, col):
    dummies = pd.get_dummies(df[col], prefix=col)
    return pd.concat([df.drop(columns=[col]), dummies], axis=1)

for col in cols:
    train = one_hot(train, col)
    test = one_hot(test, col)

# Alineación de columnas
test = test.reindex(columns=train.drop(columns=['RENDIMIENTO_GLOBAL']).columns, fill_value=0)

# =============================================
# 6. Codificación de la variable objetivo
# =============================================
mapa = {'bajo':0, 'medio-bajo':1, 'medio-alto':2, 'alto':3}
inv_mapa = {v:k for k,v in mapa.items()}

y = train['RENDIMIENTO_GLOBAL'].map(mapa)
X = train.drop(columns=['RENDIMIENTO_GLOBAL'])

# =============================================
# 7. División y entrenamiento con SVM
# =============================================
Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size=0.3, random_state=42)

clf = SVC(kernel='linear')  # Puedes probar también kernel='rbf'
clf.fit(Xtrain, ytrain)

# =============================================
# 8. Evaluación
# =============================================
yval_pred = clf.predict(Xval)
acc = accuracy_score(yval, yval_pred)
print(f"Accuracy en validación: {acc:.4f}")

scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
print(f"Accuracy CV: {scores.mean():.4f} (±{scores.std():.4f})")

# =============================================
# 9. Predicción y archivo submission
# =============================================
clf.fit(X, y)
y_pred = clf.predict(test)

submission = pd.DataFrame({
    'ID': test['ID'],
    'RENDIMIENTO_GLOBAL': pd.Series(y_pred).map(inv_mapa)
})

submission.to_csv("submission.csv", index=False)
print("Primeras filas del archivo submission:")
print(submission.head())
